---
# 0. 라이브러리
----

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# 깃 허브에 있는 데이텃뎃 다운로드를 위한 깃 클론
!git clone https://github.com/arijitx/Fewshot-Learning-with-BERT.git

# dataset이 있는 경로 지정
%cd /content/Fewshot-Learning-with-BERT/data

fatal: destination path 'Fewshot-Learning-with-BERT' already exists and is not an empty directory.
/content/Fewshot-Learning-with-BERT/data


In [3]:
!pip install torch pytorch-lightning
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[참고 사이트](https://uvadlc-notebooks.readthedocs.io/en/latest/tutorial_notebooks/tutorial16/Meta_Learning.html)

In [4]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from tqdm.notebook import tqdm
from torchvision import transforms, datasets
from sklearn.model_selection import train_test_split

from PIL import Image
from pprint import pprint
import easydict

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu') # GPU 할당

import warnings
warnings.filterwarnings(action='ignore')

import seaborn as sns 
import pandas as pd
import os
import random
import json
from collections import defaultdict
from statistics import mean, stdev
from copy import deepcopy


import matplotlib.pyplot as plt
plt.set_cmap('cividis')
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg', 'pdf') # For export
import matplotlib
matplotlib.rcParams['lines.linewidth'] = 2.0

# transformers
from transformers import BertTokenizer, BertModel
import math
import pdb

# tensorboard
# %load_ext tensorboard

# Pytorch Lightning
import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint

In [5]:
import easydict

args = easydict.EasyDict({
  'lr' : 2e-4,
  'max_len' : 110,
  'epoch' : 20,
  'N_WAY' : 5,
  'K_SHOT' : 4
})

---
# 2. Data
----

- ATIS 데이터셋을 통해 train / validation / test 셋을 구성

In [6]:
# train
text = pd.read_csv('./ATIS/train/seq.in', names = ['text'])
label = pd.read_csv('./ATIS/train/label', names = ['label'])
train_data = pd.concat([text,label], axis = 1)

# valid
text = pd.read_csv('./ATIS/valid/seq.in', names = ['text'])
label = pd.read_csv('./ATIS/valid/label', names = ['label'])
valid_data = pd.concat([text,label], axis = 1)

# test
text = pd.read_csv('./ATIS/test/seq.in', names = ['text'])
label = pd.read_csv('./ATIS/test/label', names = ['label'])
test_data = pd.concat([text,label], axis = 1)

print(train_data.shape)
print(valid_data.shape)
print(test_data.shape)

(4478, 2)
(500, 2)
(893, 2)


- label 종류 확인

In [7]:
train_label = set(train_data['label'])
valid_label = set(valid_data['label'])
test_label = set(test_data['label'])

In [8]:
print('train_set label 개수 : ', len(train_label))
print('valid_set label 개수 : ', len(valid_label))
print('test_set label 개수 : ', len(test_label))

train_set label 개수 :  21
valid_set label 개수 :  16
test_set label 개수 :  20


In [9]:
uni_label = train_label | valid_label | test_label
label_to_index = {i : idx for idx, i in enumerate(uni_label)}
index_to_label = {idx : i for idx, i in enumerate(uni_label)}

In [10]:
label_to_index

{'atis_flight#atis_airfare': 0,
 'atis_ground_fare': 1,
 'atis_day_name': 2,
 'atis_city': 3,
 'atis_flight_no': 4,
 'atis_ground_service': 5,
 'atis_cheapest': 6,
 'atis_flight_time': 7,
 'atis_airfare#atis_flight_time': 8,
 'atis_flight': 9,
 'atis_airline': 10,
 'atis_quantity': 11,
 'atis_airfare#atis_flight': 12,
 'atis_flight_no#atis_airline': 13,
 'atis_airfare': 14,
 'atis_airline#atis_flight_no': 15,
 'atis_restriction': 16,
 'atis_airport': 17,
 'atis_ground_service#atis_ground_fare': 18,
 'atis_abbreviation': 19,
 'atis_distance': 20,
 'atis_aircraft#atis_flight#atis_flight_no': 21,
 'atis_flight#atis_airline': 22,
 'atis_aircraft': 23,
 'atis_capacity': 24,
 'atis_meal': 25}

In [11]:
def trans_label(train_data):
  for i in range(len(train_data)):
    train_data['label'][i] = label_to_index[train_data['label'][i]]
  return train_data

In [12]:
trans_label(train_data)
trans_label(valid_data)
trans_label(test_data)

,text,label
0,i would like to find a flight from charlotte t...,9
1,on april first i need a ticket from tacoma to ...,14
2,on april first i need a flight going from phoe...,9
3,i would like a flight traveling one way from p...,9
4,i would like a flight from orlando to salt lak...,9
...,...,...
888,please find all the flights from cincinnati to...,9
889,find me a flight from cincinnati to any airpor...,9
890,i'd like to fly from miami to chicago on ameri...,9
891,i would like to book a round trip flight from ...,9


In [13]:
value = pd.concat([train_data, test_data, valid_data], ignore_index = True)
value

,text,label
0,i want to fly from baltimore to dallas round trip,9
1,round trip fares from baltimore to philadelphi...,14
2,show me the flights arriving on baltimore on j...,9
3,what are the flights which depart from san fra...,9
4,which airlines fly from boston to washington d...,10
...,...,...
5866,pm flights dallas to atlanta,9
5867,information on flights from baltimore to phila...,9
5868,what flights from atlanta to st. louis on tues...,9
5869,show me ground transportation in san francisco,5


In [14]:
unique_label_dict = {}
# 레이블별 개수 확인
for i in range(len(value)):
  if value['label'][i] in unique_label_dict.keys():
    unique_label_dict[value['label'][i]] += 1
  else:
    unique_label_dict[value['label'][i]] = 1

# 10개 이하인 레이블 확인
qq = []
for k, v in unique_label_dict.items():
  if v <= 10:
    qq.append(k)    

# 10개 이하인 레이블 삭제
for i in qq:
  del unique_label_dict[i]

In [15]:
dataset = pd.DataFrame()
for i in unique_label_dict.keys():
  dataset = pd.concat([dataset,value[value['label'] == i]])

dataset = dataset.reset_index(drop = True)  
unique_label_dict

{9: 4298,
 14: 471,
 10: 195,
 5: 291,
 11: 54,
 3: 25,
 0: 33,
 19: 180,
 23: 90,
 20: 30,
 1: 25,
 24: 37,
 7: 55,
 25: 12,
 4: 20,
 17: 38}

In [16]:
split_data, test_data = dataset[: -187], dataset[-187:]
train_data, valid_data = split_data[:-358], split_data[-358:]

train_data = train_data.sample(frac = 1).reset_index(drop = True)
valid_data = valid_data.sample(frac = 1).reset_index(drop = True)
test_data = test_data[:-1].sample(frac = 1).reset_index(drop = True)

print(train_data.shape) # 5개 클래스 
print(valid_data.shape) # 5개 클래스
print(test_data.shape) # 6개 클래스

(5309, 2)
(358, 2)
(186, 2)


---
# 3. Preprocessing
---

In [17]:
class BertDataset(Dataset):
  def __init__(self, data):
    self.sentence1 = []
    self.label = []
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

    for meta in data['text']:
      # BERT -> max 512 length
      self.sentence1.append(tokenizer(meta, max_length=args.max_len, padding='max_length', return_tensors='pt'))
      
    for meta in data['label']:
      self.label.append(meta)

    self.label = torch.tensor(self.label)

  def __len__(self):
    return len(self.sentence1)

  def __getitem__(self, idx):
    return self.sentence1[idx], self.label[idx]

In [18]:
BertDataset(train_data).__getitem__(0)

({'input_ids': tensor([[ 101, 2265, 2033, 1996, 7608, 2005, 7599, 2000, 2030, 2013, 2293, 2492,
           102,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [19]:
train_set = BertDataset(train_data)

valid_set = BertDataset(valid_data)

---
# 4. Data Sampling
----

In [20]:
class FewShotBatchSampler():

  def __init__(self, dataset_targets, N_way, K_shot,
               include_query = False,
               shuffle = True,
               shuffle_once = False):
    super().__init__()
  
    '''dataset_targets = 데이터 요소 레이블의 tensor
        N_way = 배치당 샘플링할 클래스 수
        K-shot = 배치에서 클래스 당 샘플링할 예제 수
        include_query = True인 경우 N_way * K-shot * 2 크기의 배치를 반환
          sup 및 qry 세트에 대해 동일한 클래스와 별개의 예제를 샘플링하는 구현을 단순화
        shuffle = True 이면 예제 및 클래스가 각 항목에서 새로 shffle
        shuffle_once = True 이면 예제와 클래스가 한 번씩 셔플 됨'''

    self.dataset_targets = dataset_targets
    self.N_way = N_way
    self.K_shot = K_shot
    self.shuffle = shuffle
    self.include_query = include_query
    if self.include_query :
      self.K_shot *= 2
    self.batch_size = self.N_way * self.K_shot

    '''batch_size를 N_way와 K_shot를 곱한 값으로 하는 이유는??
          
          N-way K-shot classification 문제를 해결하기 위해
          
          N-way K-shot문제란 N 개의 클래스 중 K개의 샘플만 사용하여 학습하는 분류 문제인데 
          N = 클래스의 수 / K = 각 클래스 마다 사용되는 샘플의 개수 
          N_way = 한 배치당 몇 개의 클래스르 사용할지 결정하는 변수
          K_shot = 각 클래스 당 몇 개의 샘플을 사용할지 결정하는 변수
          
          예를 들어  N_way = 5, K_shot = 1인 경우, 한 배치는 5개의 클래스를 가지며, 각 클래스당 1개의 샘플을 사용하여
          총 5 * 1 = 5개의 샘플이 한 배치를 이루게 된다.
          따라서 N_way와 K_shot를 곱하는 이뉴는 
          한 배치에서 사용할 이미지의 개수를 결정하기 위해서 이다.'''
    
    ############################# 클래스별 예제 구성 #############################
    # 고유한 클래스 라벨 추출
    self.classes = torch.unique(self.dataset_targets).tolist()
    
    # 클래스 총 개수 추출
    self.num_classes = len(self.classes)

    # 클래스 별 데이터셋의 인덱스
    self.indices_per_class = {}
    
    # 클래스 별 데이터셋의 K_shot 배치 개수
    self.batches_per_class = {} 

    # 각 클래스 별로 데이터셋의 인덱스를 추출
    for i in self.classes:
      self.indices_per_class[i] = torch.where(self.dataset_targets == i)[0]
      
      # 해당 클래스에서 만들 수 있는 k-shot 배치의 개수를 의미
      self.batches_per_class[i] = self.indices_per_class[i].shape[0] // self.K_shot
    ##########################################################################

    ############################# batch 당 N 개의 클래스를 선택하는 클래스 목록 구성 #############################
    # 각 클래스 마다 K-shot개의 이미지로 이루어진 배치를 몇 번 만들 수 있는지 계산해, 총 몇 번의 iteration이 필요한지 계산
    self.iterations = sum(self.batches_per_class.values()) // self.N_way

    # 각 iteration에서 선택할 N_way개의 클래스를 저장 / 각 클래스에서 생성 가능한 배치 수 만큼 클래스를 반복하여 리스트에 추가
    self.class_list = [c for c in self.classes for _ in range(self.batches_per_class[c])]

    # True일 경우 shuffle_data를 호출하여 클래스 리스틀 한 번 Shuffle
    if shuffle_once or self.shuffle:
      self.shuffle_data()
    
    # False일 경우 = 데이터셋을 test 할 경우를 의미
    # 클래스를 섞는 대신, 정해진 순서대로 iteration을 진행해야 하므로, 이를 위해 인덱스 리스트를 계산해 클래스 리스트를 순서대로 정렬(np.argsort(sort_list))
    else:
      sort_idxs = [i + p * self.num_classes for i, c in enumerate(self.classes) for p in range(self.batches_per_class[c])]
      self.class_list = np.array(self.class_list)[np.argsort(sort_idxs)].tolist()
    ######################################################################################################

  def shuffle_data(self):  # 각 배치에서 다른 클래스의 데이터를 섞어 overfitting을 방지하고 학습을 일반화 시킴
    
    # 클래스 당 예제별 shuffle 
    for c in self.classes:      
      # indices_per_class[c]에서 원소의 인덱스를 섞는다
      perm = torch.randperm(self.indices_per_class[c].shape[0])
      
      # 위에서 섞인 perm의 인덱스 순서에 따라 indices_per_class[c]의 원소들을 섞는다
      self.indices_per_class[c] = self.indices_per_class[c][perm]
    
    # 모든 클래스를 포함한 classes_list의 순서를 랜덤하게 섞는다
    random.shuffle(self.class_list)

  def __iter__(self):
    
    # Shuffle data
    if self.shuffle:
      self.shuffle_data()

    #################### few-shot 배치 샘플 ####################
    # 딕셔너리의 기본값을 0으로 설정
    start_index = defaultdict(int)
    
    for i in range(self.iterations):
      # N_way의 개수 만큼 클래스를 class_batch에 할당
      class_batch = self.class_list[i * self.N_way : (i + 1) * self.N_way]
      index_batch = []
      for c in class_batch:
        # 클래스 c 에서 K-shot 만큼 인덱스를 선택하여 index_batch에 추가
        index_batch.extend(self.indices_per_class[c][start_index[c] : start_index[c] + self.K_shot])
        # 다음 반복을 위해 K-shot 만큼 건너뛰어 선택하도록
        start_index[c] += self.K_shot
      
      # Query_set을 포함하도록 선택하였다면, sup_set과 qry_set을 번갈아 가면서 배치
      if self.include_query:
        index_batch = index_batch[::2] + index_batch[1::2]

      # minibatch를 구성하는 데이터 샘플들의 인덱스 리스트 반환
      yield index_batch
    ###########################################################

  def __len__(self):
    return self.iterations

In [21]:
# 기존 FewShotBatchSampler 코드에 더해서 ProtoMAML 샘플링에 사용될 코드 추가 
class TaskBatchSampler():

  def __init__(self, dataset_targets, batch_size, N_way, K_shot, include_query = False, shuffle = True):
    super().__init__()
    '''dataset_targets = 데이터 요소 레이블의 Tensor
       batch_size = 배치 사이즈
       N_way = task 내에서 배치당 샘플링할 클래스 수
       K_shot = 배치에서 클래스당 샘플링할 예제 수
       include_query = True이면 sup, qry 세트로 분할 할 수 있는 N_way * K_shot * 2 크기의 배치를 반환
       shuffle = True이면 각 반복에서 예제와 클래스가 새로 셔플됨 (Train용)
    '''

    self.batch_sampler = FewShotBatchSampler(dataset_targets, N_way, K_shot, include_query, shuffle)
    self.task_batch_size = batch_size
    self.local_batch_size = self.batch_sampler.batch_size

  # class를 반복 가능하도록 만들기 위한 함수
  def __iter__(self):

    batch_list = []
    
    # batch_sampler를 통해 데이터 배치를 샘플링
    for batch_idx, batch in enumerate(self.batch_sampler):
      batch_list.extend(batch)

      # 만약 task_batch_size 개수 만금 배치가 쌓이면 batch_list를 반환 하고 초기화
      if (batch_idx + 1) % self.task_batch_size == 0:
        yield batch_list
        batch_list = []

  def __len__(self):
    return len(self.batch_sampler) // self.task_batch_size

  # 작업별 tensor 목록으로 변환하는 collate_fn을 반환하는 코드
  def get_collate_fn(self):
    
    '''get_collate_fn 함수 안에 정의된 collate_fn 함수는 get_collate_fn 함수의 지역 함수로 정의한다. 
       이렇게 함수 안에 또 다른 함수를 정의하는 것을 "내부 함수" 또는 "지역 함수"라고 부른다.

       collate_fn 함수는 get_collate_fn 함수와 같은 레벨에서 정의된 변수와 메소드에 모두 접근할 수 있다. 
       이 경우 self.task_batch_size와 같은 TaskBatchSampler 클래스의 인스턴스 변수에 접근할 수 있다. 
       이는 collate_fn 함수를 외부에서 독립적으로 정의하는 것보다 더 효율적이며 코드 가독성도 높아진다.'''
    
    # Dataloader에서 각 배치마다 호출되어 해당 배치의 데이터를 처리하기 위한 함수
    def collate_fn(batch):

      # batch: List of BatchEncoding objects
      _input_ids = [b[0]['input_ids'] for b in batch]
      _attention_mask = [b[0]['attention_mask'] for b in batch]
      _token_type_ids = [b[0]['token_type_ids'] for b in batch]
      _targets = [b[1] for b in batch]

      # Stack the lists of tensors
      input_ids = torch.stack(_input_ids, dim=0)
      attention_mask = torch.stack(_attention_mask, dim=0)
      token_type_ids = torch.stack(_token_type_ids, dim=0)
      targets = torch.stack(_targets, dim=0)

      input_ids = input_ids.chunk(self.task_batch_size, dim = 0)
      attention_mask = attention_mask.chunk(self.task_batch_size, dim = 0)
      token_type_ids = token_type_ids.chunk(self.task_batch_size, dim = 0)
      targets = targets.chunk(self.task_batch_size, dim = 0)

      # Return a tuple of tensors
      return list(zip(input_ids, attention_mask, token_type_ids, targets))
    
    return collate_fn
    

In [22]:
N_WAY = args.N_WAY
K_SHOT = args.K_SHOT

# Training set
train_protomaml_sampler = TaskBatchSampler(train_set.label,
                                           include_query=True,
                                           N_way=N_WAY,
                                           K_shot=K_SHOT,
                                           batch_size=16)

train_protomaml_loader = torch.utils.data.DataLoader(train_set,
                                         batch_sampler=train_protomaml_sampler,
                                         collate_fn=train_protomaml_sampler.get_collate_fn(),
                                         num_workers=0)

# Validation set
val_protomaml_sampler = TaskBatchSampler(valid_set.label,
                                         include_query=True,
                                         N_way=N_WAY,
                                         K_shot=K_SHOT,
                                         batch_size=1,  # parameter를 업데이트 하지 않기 때문에 배치 사이즈는 무관
                                         shuffle=False)

val_protomaml_loader = torch.utils.data.DataLoader(valid_set,
                                       batch_sampler=val_protomaml_sampler,
                                       collate_fn=val_protomaml_sampler.get_collate_fn(),
                                       num_workers=0)

In [23]:
def split_batch_bert(imgs, targets):

  sup_a , qry_a = imgs[0].squeeze(1).chunk(2, dim = 0)
  sup_b , qry_b = imgs[1].squeeze(1).chunk(2, dim = 0)
  sup_c , qry_c = imgs[2].squeeze(1).chunk(2, dim = 0)

  support_targets, query_targets = targets.chunk(2, dim=0)
  
  support_imgs = {'input_ids' : sup_a, 'token_type_ids' : sup_b, 'attention_mask' : sup_c}
  query_imgs = {'input_ids' : qry_a, 'token_type_ids' : qry_b, 'attention_mask' : qry_c}

  return support_imgs, query_imgs, support_targets, query_targets

In [24]:
def split_batch(imgs, targets):

  support_imgs, query_imgs = imgs.chunk(2, dim=0)
  support_targets, query_targets = targets.chunk(2, dim=0)
  return support_imgs, query_imgs, support_targets, query_targets

- sup_set과 qry_set 에는 동일한 5개의 클래스가 존재하지만 예제는 다르게 나타난다.

- 모델은 sup_set 및 해당 레이블에서 학습하여 qry_set의 예를 분류하는 task를 수행하게 된다. 

---
# 5. Prototype Networks
---

In [25]:
class BERT(nn.Module):
	def __init__(self,  n_input = 768, n_output = 128, bert_model = 'bert-base-uncased'):
		super(BERT,self).__init__()
		self.bert = BertModel.from_pretrained(bert_model)

	def forward(self, input_ids, attention_mask, token_type_ids):
		all_hidden_layers, _ = self.bert(input_ids = input_ids, 
                                     attention_mask = attention_mask,
                                     token_type_ids = token_type_ids, return_dict = False)
		cls_hn = all_hidden_layers[:,0,:]
		return cls_hn

In [26]:
class ProtoNet(pl.LightningModule):

  def __init__(self, lr):
    super().__init__()

    self.model = BERT()
    self.save_hyperparameters()
    self.lr = lr

  def configure_optimizers(self):
    optimizer = optim.AdamW(self.model.parameters(), lr = self.lr)
    scheduler = optim.lr_scheduler.MultiStepLR(
          optimizer, milestones = [20, 40], gamma = 0.1)
    
    return [optimizer], [scheduler]

  ################# few-shot classification에서 사용될 Prototype을 계산하는 함수 #################
  @staticmethod
  def calculate_prototypes(features, targets):
    # 정답 레이블에서 고유한 클래스 레이블을 찾아 리스트에 저장 및 정렬
    classes, _ = torch.unique(targets).sort()
    # 프로토타입을 저장할 빈 리스트
    prototypes = []
    for i in classes : # 클래스 라벨 i를 반복
      # 현재 클래스 i에 속하는 샘플들의 특징 벡터를 선택해 평균을 계산
      p = features[torch.where(targets == i)[0]].mean(dim = 0)
      prototypes.append(p)
    
    # prototypes 리스트를 tensor로 변환하여 반환 (dim = 0는 행으로 스택)
    prototypes = torch.stack(prototypes, dim = 0)

    return prototypes, classes
  
  ############## 새로운 샘플을 프로토타입으로 분류하고 분류 오류를 반환하는 함수 ##############
  def classify_feats(self, prototypes, classes, feats, targets):

    # 두 벡터 사이의 유클리드 거리(2- 제곱 거리)를 계산
    dist = torch.pow(prototypes[None, :] - feats[:, None], 2).sum(dim = 2)
    '''prototype과 feats는 각각 K x D 크기의 Tensor
       k = 클래스의 수 / D = 특징 벡터
       Prototype[None, :] .> [1 x K x D] 
       feats[:, None] -> [N x 1 x D]
       두 tensor를 뺀 결과는 [N x K x D] 이며 제곱을 하고 마지막 차원을 따라 더하면 [N x K]의 형태가 나옴'''
  
    # 클래스에 대한 로그 확률을 계산 
    preds = F.log_softmax( - dist, dim = 1)

    # 각 행에서 가장 높은 값을 가지는 열의 인덱스를 찾아 반환
    labels = (classes[None, :] == targets[:, None]).long().argmax(dim=1)
    '''classes = 모든 클래스 ID를 포함하는 tensor
       targets = 각 샘플의 실제 클래스 ID를 포함하는 tensor
       첫 번째 차원에 대해 브로드캐스팅하여 [N x K]의 tensor'''
  
    # preds과 labels 값의 분류 정확도 계산
    acc = (preds.argmax(dim=1) == labels).float().mean()

    return preds, labels, acc

  ################### 현재 모델과 배치를 기반으로 손실을 계산하는 함수 ###################
  def calculate_loss(self, batch, mode):
    
    text, label = batch

    # 모델을 통해 인코딩된 이미지 특성 벡터 생성 (sup_set과 qry_set)
    features = self.model(text['input_ids'].squeeze(1),
                          text['attention_mask'].squeeze(1),
                          text['token_type_ids'].squeeze(1))

    # sup_set의 특성 벡터, qry_set의 특성 벡터, sup_set의 레이블, qry_set의 레이블 계산
    support_feats, query_feats, support_targets, query_targets = split_batch(features, label)

    # sup_set에서 각 클래스의 Prototype을 계산
    prototypes, classes = ProtoNet.calculate_prototypes(support_feats, support_targets)

    # qry_set에 대해서 Prototype을 이용해 분류를 수행하고 예측 결과, 실제 레이블, 정확도 계산
    preds, labels, acc = self.classify_feats(prototypes, classes, query_feats, query_targets)

    # loss 계산
    loss = F.cross_entropy(preds, labels)

    # 모델의 log 저장
    self.log(f"{mode}_loss", loss)
    self.log(f"{mode}_acc", acc)
    
    return loss

  def training_step(self, batch, batch_idx):
      return self.calculate_loss(batch, mode="train")

  def validation_step(self, batch, batch_idx):
      _ = self.calculate_loss(batch, mode="val")

In [27]:
class ProtoMAML(pl.LightningModule):

  def __init__(self, lr, lr_inner, lr_output, num_inner_steps):
    super().__init__()
    '''lr_inner = 내부 루프의 학습 속도(SGD)
       lr_output = 내부 루프의 output layer에 대한 학습 속도
       num_inner_steps = 수행할 inner-loop epoch 수'''
       
    self.save_hyperparameters()
    self.model = BERT()

  def configure_optimizers(self):
    optimizer = optim.AdamW(self.parameters(), lr = self.hparams.lr)
    scheduler = optim.lr_scheduler.MultiStepLR( optimizer, milestones = [20, 40], gamma = 0.1)
    
    return [optimizer], [scheduler]

  # 지정된 출력 레이어 weight 및 input을 사용하여 모델 실행
  def run_model(self, local_model, output_weight, output_bias, imgs, labels):
    feats = local_model(imgs['input_ids'].squeeze(1),
                        imgs['attention_mask'].squeeze(1),
                        imgs['token_type_ids'].squeeze(1))

    preds = F.linear(feats, output_weight, output_bias)
    
    loss = F.cross_entropy(preds, labels)
    
    acc = (preds.argmax(dim = 1) == labels).float()

    return loss, preds, acc 

  def adapt_few_shot(self, support_imgs, support_targets):
    
    # feature를 얻기 위해 기본 모델(DenseNet) 통과
    support_feats = self.model(support_imgs['input_ids'].squeeze(1),
                               support_imgs['attention_mask'].squeeze(1),
                               support_imgs['token_type_ids'].squeeze(1))
    
    # 추출된 feature와 label은 프로토타입 계산에 사용
    prototypes, classes = ProtoNet.calculate_prototypes(support_feats, support_targets)

    support_labels = (classes[None, :] == support_targets[:, None]).long().argmax(dim = -1)
    '''classes = sup_set의 클래스 인덱스를 담은 벡터
       support_targets = sup_set의 타겟 레이블을 담은 벡터
       
       브로드캐스팅을 위해 각 차원을 변경하고 둘을 비교해 같은 클래스에 속하는 원소를 1 또는 0으로 표시한 행렬 생성
       즉, support_labels는 k개의 sup_set이 어떤 클래스에 속하는지 나타내는 차원의 벡터
       마지막으로 argmax를 통해 sup_set가 속한 클래스 중 가장 높은 확률을 갖는 클래스 인덱스를 구한다.'''

    #################### inner-loop 모델 생성 ####################
    local_model = deepcopy(self.model) # 기본 모델의 로컬 복사본 생성
    local_model.train()
    local_optim = optim.SGD(local_model.parameters(), lr = self.hparams.lr_inner)
    local_optim.zero_grad()
    ############################################################


    ################## 프로토타입 기반 초기화를 사용하여 출력 계층 가중치 생성 ##################
    # init weight 및 bias는 프로토타입의 tensor와 동일한 모양의 tensor로 초기화
    init_weight = 2 * prototypes
    init_bias = -torch.norm(prototypes, dim=1)**2
    '''초기 weight를 2 * prototype으로 하는 이유?
    
       ProtoNet에서 분류 작업을 하기 위한 최종 output layer에서는 각 클래스에 해당하는 프로토타입과 유사도를 계산하여 사용한다. 
       이때, 프로토타입들과의 유사도를 계산하기 위해서는 최종 output layer의 weight가 프로토타입들의 값과 일치해야한다.
       
       초기 weight를 2 * prototype로 설정함으로써, 모델이 학습을 거듭하면서 최종 output layer의 weight가 조금씩 수정되어가는 과정에서도
       항상 프로토타입 값과 유사하게 유지될 가능성이 높아지기 때문이다.'''

    # output weight 및 bias는 각 init의 복사본으로 초기화
    output_weight = init_weight.detach().requires_grad_()
    output_bias = init_bias.detach().requires_grad_()
    '''output weight와 bias는 초기화가 필요하지 않은 완성된 값이다.
       inner-loop를 돌면서 계산된 output W&B와 init W&B를 더하여 최종 W&B를 얻게된다.
       
       따라서 output W&B가 반복적으로 초기화되어야 하며 
       이렇게 하면 inner-loop에서 새로운 W&B가 업데이트 될때 이전 W&B가 유지되게 된다.
       (초기화된 W&B이 다음 step에서도 유지되어야 이전에 결정된 프로토타입에 대한 정보를 유지하고 더 나은 few-shot learning을 할 수 있게 된다.)'''

    # support_set을 사용한 inner-loop 모델
    for _ in range(self.hparams.num_inner_steps):

      loss, _, _ = self.run_model(local_model, output_weight, output_bias, support_imgs, support_labels)
      
      # Gradient 계산 및 inner-loop 업데이트
      loss.backward(retain_graph=True)
      local_optim.step()

      # SGD를 이용한 output Layer 업데이트
      output_weight.data -= self.hparams.lr_output * output_weight.grad
      output_bias.data -= self.hparams.lr_output * output_bias.grad

      # Gradient 초기화
      local_optim.zero_grad()
      output_weight.grad.fill_(0)
      output_bias.grad.fill_(0)

    # 프로토타입의 계산 그래프 다시 첨부
    output_weight = (output_weight - init_weight).detach() + init_weight
    output_bias = (output_bias - init_bias).detach() + init_bias
    '''이전에 초기화한 값(init_weight, init_bias)과 조정된 값을(output_weight, output_bias)을 빼준 다음, 
       detach() 메소드를 사용하여 연산 결과를 새로운 Tensor로 만든다. 
       이렇게 함으로써, 계산된 gradient가 초기화된 값에 영향을 미치지 않도록 방지할 수 있다. 
       
       마지막으로, 다시 이전 초기화한 값에 더해줌으로써, 
       최종 출력 레이어의 W&B가 초기화한 값에서 적절하게 조정된 값으로 업데이트된다.'''

    return local_model, output_weight, output_bias, classes


  def outer_loop(self, batch, mode = 'train'):

    accuracies = []
    losses = []
    self.model.zero_grad()
    
    # 배치에 대한 Gradient 결정
    for task_batch in batch:
      
      imgs, targets = [task_batch[0], task_batch[1], task_batch[2]], task_batch[3]
      
      # 각 task에 대해서 sup, qry 분리
      support_imgs, query_imgs, support_targets, query_targets = split_batch_bert(imgs, targets)

      # inner-loop에 적용
      local_model, output_weight, output_bias, classes = self.adapt_few_shot(support_imgs, support_targets)
      
      # query set의 손실값 확인
      query_labels = (classes[None,:] == query_targets[:,None]).long().argmax(dim=-1)
      loss, preds, acc = self.run_model(local_model, output_weight, output_bias, query_imgs, query_labels)

      # query_set 손실값에 대한 Gradient 계산
      if mode == 'train':
        loss.backward()
        
        # for p_global, p_local in zip(self.model.parameters(), local_model.parameters()):
        #   print(p_global.grad)
        #   print(p_local.grad)
        #   p_global.grad += p_local.grad
        for p_global, p_local in zip(self.model.parameters(), local_model.parameters()):
            if p_local.grad is not None:
                if p_global.grad is None:
                    p_global.grad = torch.zeros_like(p_local.grad)
                p_global.grad += p_local.grad

      accuracies.append(acc.mean().detach())
      losses.append(loss.detach())

    if mode == 'train':
      opt = self.optimizers()
      opt.step()
      opt.zero_grad()

    self.log(f"{mode}_loss", sum(losses) / len(losses))
    self.log(f"{mode}_acc", sum(accuracies) / len(accuracies))

  def training_step(self, batch, batch_idx):
    self.outer_loop(batch, mode="train")
    return None
    '''왜 None을 반환하는가?
    
       원래는 학습 단계를 실행하고 최종적으로 loss 값을 반환해야 하나
       outer_loop 함수에서 loss 값을 계싼하고 역전파를 수행하기 때문에 
       여기서는 None을 반환한다.'''

  def validation_step(self, batch, batch_idx):
    
    # 검증 단계에서는 모델을 미세 조정해야 하므로 Gradient를 사용하도록 설정 -> 모델의 일반화 성능을 더 향상 시키기 위해
    torch.set_grad_enabled(True)
    self.outer_loop(batch, mode="val")
    torch.set_grad_enabled(False)



---
# 6. Train
----

In [28]:
CHECKPOINT_PATH = '/content/drive/MyDrive/2.Study/Few-Shot/'

def train_model(model_class, train_loader, val_loader, **kwargs):

  trainer = pl.Trainer(default_root_dir = os.path.join(CHECKPOINT_PATH, model_class.__name__),
                       accelerator = "gpu" if str(device).startswith("cuda") else "cpu",
                       devices = 1,
                       max_epochs = args.epoch,
                       callbacks = [ModelCheckpoint(save_weights_only=True, mode="max", monitor="val_acc"),
                                    LearningRateMonitor("epoch")],
                       enable_progress_bar = True)
  '''default_root_dir = 체크포인트 파일이 저장될 경로
     accelerator = CPU 또는 GPU
     devices = 학습에 사용할 디바이스 개수
     max_epochs = 학습 에폭 수
     callbacks = 학습 도중에 호출될 콜백 목록, 체크포인트 저장, 모니터링 여부 등
     enable_progress_bar = 진행 상황 표시 여부'''
  trainer.logger._default_hp_metric = None

  # 사전에 훈련한 체크 포인트가 있는지 확인하고 있다면 연결, 없다면 새로 학습
  pretrained_filename = os.path.join(CHECKPOINT_PATH, model_class.__name__  + '.ckpt')

  if os.path.isfile(pretrained_filename):
    print(f"Found pretrained model at {pretrained_filename}, loading...")
    # 자동적으로 모델 및 하이퍼파라미터 로드
    model = model_class.load_from_checkpoint(pretrained_filename)
  else:
    # 랜덤 시드를 고정하여 재현성 보장
    pl.seed_everything(42)
    
    # 주어진 모델 클래스와 인자로 부터 모델 객체를 생성
    model = model_class(**kwargs)
    
    # 모델 학습 
    trainer.fit(model, train_loader, val_loader)

    # 학습이 완료된 후 체크포인트 콜백에서 가장 좋은 성능을 보인 모델의 체크포인트 파일을 로드하요 모델 생성
    model = model_class.load_from_checkpoint(
              trainer.checkpoint_callback.best_model_path)
    
  return model

- 학습 진행

In [29]:
protomaml_model = train_model(ProtoMAML,
                              lr=1e-3,
                              lr_inner=0.1,
                              lr_output=0.1,
                              num_inner_steps=1,  # Often values between 1 and 10
                              train_loader=train_protomaml_loader,
                              val_loader=val_protomaml_loader)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:lightning_fabric.utilities.seed:Global seed set to 42
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=20` reached.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


---
# 7. Test
----

In [30]:
test_set = BertDataset(test_data)

protomaml_model.hparams.num_inner_steps = 200

In [31]:
def test_protomaml(model, dataset, shot):
  '''model = 사전 학습된 ProtoNet Model
      dataset = test_dataset 사용
      data_feats = 이미지 특징 데이터
      K_shot = 샷의 수'''
  pl.seed_everything(42)
  model = model.to(device)

  # 데이터셋에서 고유한 클래스 수 계산
  num_classes = dataset.label.unique().shape[0]   # num_classes = 6

  # 각 클래스 당 이미지 샘플 수 계산
  exmps_per_class = dataset.label.shape[0]//num_classes # exmps_per_class = 31 

  # DataLoader 생성
  full_dataloader = torch.utils.data.DataLoader(dataset,
                                                batch_size=16,
                                                num_workers=0,
                                                shuffle=False,
                                                drop_last=False)

  # few-shot을 위해 k-shot 개수 만큼 샘플을 뽑아 DataLoader 생성
  sampler = FewShotBatchSampler(dataset.label,
                                include_query=False,
                                N_way=num_classes,
                                K_shot=shot,
                                shuffle=False,
                                shuffle_once=False)

  sample_dataloader = torch.utils.data.DataLoader(dataset,
                                                  batch_sampler=sampler,
                                                  num_workers=1)


  # 먼저 k-shot 배치를 선택하고, 다른 모든 예제에서 모델을 평가
  accuracies = []
  for value, support_indices in tqdm(zip(sample_dataloader, sampler), "Performing few-shot finetuning"):
    support_imgs = value[0].to(device)
    support_targets = value[1].to(device)

    # 샘플 데이터에 대한 fine-tuning을 진행하고, local_model과 프로토타입을 반환
    local_model, output_weight, output_bias, classes = model.adapt_few_shot(support_imgs, support_targets)

    with torch.no_grad():  
      # qry_set에 대한 역전파 불필요
      local_model.eval()
      batch_acc = torch.zeros((0,), dtype=torch.float32, device=device)
      
      # 테스트 데이터 세트의 모든 예제 평가 
      for query in full_dataloader:
        query_imgs = query[0].to(device)
        query_targets = query[1].to(device)
        query_labels = (classes[None,:] == query_targets[:,None]).long().argmax(dim=-1)

        
        # (loss), (preds), acc 출력
        _, _, acc = model.run_model(local_model, output_weight, output_bias, query_imgs, query_labels)
        batch_acc = torch.cat([batch_acc, acc.detach()], dim=0)
    
      # sup_set 요소 제외
      for s_idx in support_indices:
        batch_acc[s_idx] = 0
      batch_acc = batch_acc.sum().item() / (batch_acc.shape[0] - len(support_indices))
      accuracies.append(batch_acc)
          
  return mean(accuracies), stdev(accuracies)

In [32]:
protomaml_accuracies = dict()
for k in [2, 4, 8, 16, 32]:
    protomaml_accuracies[k] = test_protomaml(protomaml_model, test_set, shot=k)
    print(f"Accuracy for k={k}: {100.0*protomaml_accuracies[k][0]:4.2f}% (+-{100.0*protomaml_accuracies[k][1]:4.2f}%)", '\n')    

INFO:lightning_fabric.utilities.seed:Global seed set to 42


Performing few-shot finetuning: 0it [00:00, ?it/s]

INFO:lightning_fabric.utilities.seed:Global seed set to 42


Accuracy for k=2: 28.20% (+-23.25%) 



Performing few-shot finetuning: 0it [00:00, ?it/s]

INFO:lightning_fabric.utilities.seed:Global seed set to 42


Accuracy for k=4: 18.34% (+-7.10%) 



Performing few-shot finetuning: 0it [00:00, ?it/s]

OutOfMemoryError: ignored

In [ ]:
def plot_few_shot(acc_dict, name, color=None, ax=None):
    sns.set()
    if ax is None:
        fig, ax = plt.subplots(1,1,figsize=(5,3))
    ks = sorted(list(acc_dict.keys()))
    mean_accs = [acc_dict[k][0] for k in ks]
    std_accs = [acc_dict[k][1] for k in ks]
    ax.plot(ks, mean_accs, marker='o', markeredgecolor='k', markersize=6, label=name, color=color)
    ax.fill_between(ks, [m-s for m,s in zip(mean_accs, std_accs)], [m+s for m,s in zip(mean_accs, std_accs)], alpha=0.2, color=color)
    ax.set_xticks(ks)
    ax.set_xlim([ks[0]-1, ks[-1]+1])
    ax.set_xlabel("Number of shots per class", weight='bold')
    ax.set_ylabel("Accuracy", weight='bold')
    if len(ax.get_title()) == 0:
        ax.set_title("Few-Shot Performance " + name, weight='bold')
    else:
        ax.set_title(ax.get_title() + " and " + name, weight='bold')
    ax.legend()
    return ax

In [ ]:
ax = plot_few_shot(protomaml_accuracies, name="ProtoMAML", color="C1")
plt.show()
plt.close()

---
# 8. Another Dataset Adapt
---

In [ ]:
# test
text = pd.read_csv('./SNIPS/test/seq.in', names = ['text'])
label = pd.read_csv('./SNIPS/test/label', names = ['label'])
another_data = pd.concat([text,label], axis = 1)

In [ ]:
another_set = BertDataset(another_data)

In [ ]:
protomaml_another_accuracies = dict()

for k in [2, 4, 8, 10, 12, 14, 16]:
    protomaml_another_accuracies[k], data_feats = test_protomaml(protomaml_model, another_set, shot=k)
    print(f"Accuracy for k={k}: {100.0*protomaml_accuracies[k][0]:4.2f}% (+-{100*protomaml_accuracies[k][1]:4.2f}%)", '\n')

In [ ]:
ax = plot_few_shot(protomaml_another_accuracies, name="ProtoMAML", color="C1")
plt.show()
plt.close()